In [1]:
import requests
import json
from pandas import DataFrame
import os

In [2]:
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

In [3]:
query = '여름 패션'
page = 1
size = 80
key ="9d66444b8a4cae09e6444e1c52f5a57e"
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"


In [4]:
session = requests.Session()
session.headers.update({
    "Authorization": "KakaoAK %s" % key
})

In [5]:
isEnd = False

page = 1

count = 0

while (not isEnd):
    url = urlFmt.format(query=query, page=page, size=size)
    url
    r = session.get(url)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    
    r.encoding = "utf-8"

    data = json.loads(r.text)
    data
    searchList = data['documents']
    print(searchList)
    break
    isEnd = bool(data['meta']['is_end'])
    page +=1
    print('isEnd=', isEnd, '/ 다음페이지=', page)
    
    if os.path.exists(query) == False:
        os.mkdir(query)
        
    for i, v in enumerate(searchList):
        count += 1
        docUrl = v['doc_url']
        imageUrl = v['image_url']
       
        session.headers.update({
            "Authorization": "KakaoAK %s" %key,
            "Referer": docUrl,
            "User-Agent": userAgent

        })
        try:
            r = session.get(imageUrl, stream=True)
        except:
            continue
        if r.status_code != 200:
            continue

        fileName ="%s%05d.png" % (query,count)
        with open('%s/%s' % (query,fileName), "wb") as f:
            f.write(r.raw.read())
            print(fileName,'(이)가 저장되었습니다.')
    if page == 3:
        break
       
print('종료')
    

[{'collection': 'cafe', 'datetime': '2010-07-06T19:21:19.000+09:00', 'display_sitename': 'Daum카페', 'doc_url': 'https://cafe.daum.net/hhj21/A9Ub/28', 'height': 499, 'image_url': 'http://cfile267.uf.daum.net/image/124C0A224C2D87C046479F', 'thumbnail_url': 'https://search3.kakaocdn.net/argon/130x130_85_c/Jg11sASU9oj', 'width': 640}, {'collection': 'blog', 'datetime': '2016-06-06T19:13:00.000+09:00', 'display_sitename': '네이버블로그', 'doc_url': 'http://blog.naver.com/luomoclub/220729118175', 'height': 546, 'image_url': 'https://postfiles.pstatic.net/20160606_234/luomoclub_1465207242376SyuGK_PNG/%A9%A7o8.png?type=w1', 'thumbnail_url': 'https://search3.kakaocdn.net/argon/130x130_85_c/EJckvNGksAL', 'width': 483}, {'collection': 'blog', 'datetime': '2012-06-14T14:37:00.000+09:00', 'display_sitename': '네이버블로그', 'doc_url': 'http://blog.naver.com/haru1346/100160264926', 'height': 434, 'image_url': 'https://postfiles.pstatic.net/20120614_135/haru1346_1339650177440kagG0_JPEG/naver_com_20120614_134627.j